In [8]:
### NEW CLASS CALLED PGEN CREATED ###
#### I HAVENT OPTIMIZED THIS BECAUSE IM NOT FEELING TOO WELL, BUT ILL GET ON IT TOMORROW ####
#### SO ESSENTIALLY IN THIS CASE INSTEAD OF USING WORD2VEC WE ARE GENERATING OUR EMBEDDINGS DIRECTLY USING THE NN.EMBEDDING MODULE ######
#### THE PAPER DID A SIMILAR THING #####
import numpy as np
import torch.nn as nn
import torch
import json
import fastai
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import json 
import os
import pickle
import requests
from torch.nn.utils.rnn import pad_sequence

class PGen(nn.Module):

    def __init__(self, embedding_dims, hidden_size, dropout,vocab_size,input_size,num_layers):
        super(PGen, self).__init__()

        # Hyper-parameters
        self.embedding_dims = embedding_dims
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cross_entropy = nn.CrossEntropyLoss()
        self.mse = nn.MSELoss()
        self.dropout = dropout
        self.vocab_size = vocab_size
        self.input_size = input_size
        self.hsize_decoder = int(self.hidden_size/2)
        # Encoder
        self.embedding = nn.Embedding(self.vocab_size,self.embedding_dims,sparse=False) # Lookup table with word vectors and word index in vocab
#         self.linear = nn.Sequential(nn.Linear(input_size,linear_output_size),nn.Relu(),nn.Linear(linear_output_size,input_size))
        self.lstm_encoder = nn.GRU(self.embedding_dims, self.hidden_size, self.num_layers, batch_first=True, bidirectional = False,dropout = self.dropout) # LSTM module
        self.encoder = nn.Sequential(self.embedding,self.lstm_encoder)
        
        # Decoder
        self.linear_out = nn.Linear(in_features = self.hsize_decoder,out_features = self.input_size,bias = False)
        self.lstm_decoder = nn.LSTM(self.hidden_size, self.hsize_decoder, self.num_layers, bidirectional = False, dropout = self.dropout,batch_first = True) 
        self.decoder_softmax = nn.Softmax(dim=-1)
        
        # Final Module
        self.lstm_linear_decoder = nn.Sequential(self.lstm_decoder)
        self.final_layer_decoder = nn.Sequential(self.linear_out,self.decoder_softmax)
        
        self.init_weights()

    def local_loss_contrib(self, q_hat, q_real): ### MAKE SURE THIS IS CORRECT
        ### QUESTION: In the paper they have, what looks like, average across text, right?
        sentence_length = q_hat.shape[0]
        s = torch.log(q_hat)*q_real
#         s = 0
#         epsilon = 0.001
#         for i in range(q_hat.shape[0]):
#             dot = torch.dot(torch.log(q_hat[i,:]+epsilon), q_real[i,:])
#             s += dot
# #             print(torch.log(q_hat[i,:]+epsilon),q_hat[i,:])
        return torch.abs((1/sentence_length)*s)

    def init_weights(self): ##
        # For the encoder
        init.orthogonal_(self.lstm_encoder.weight_ih_l0)
        init.uniform_(self.lstm_encoder.weight_hh_l0, a=-0.01, b=0.01) # Values taken from the paper
        # For the decoder
        init.orthogonal_(self.lstm_decoder.weight_ih_l0)
        init.uniform_(self.lstm_decoder.weight_hh_l0, a=-0.01, b=0.01) 
        
    
    def forward(self, sentence, q_real,indices_given=True): #### GET THIS TO WORK ####
        """
        A forward pass on the encoder and decoder module. Input and ground truth come in as matrix of indicies, later embedded to size [sentence length, batch size, embedding dims]
        """

        # Initialise hidden layer and cell state, size = [num_layers*num_directions, batch, hidden_size]
        print("Hi")
        _,encoded_sentence = self.encoder(sentence) # PROBLEMS COMING HERE
        print(encoded_sentence.shape)
        encoded_sentence = torch.reshape(encoded_sentence,(bs,num_layers,hidden_size))
        print(encoded_sentence.shape)
        decoded_mid,_ = self.lstm_linear_decoder(encoded_sentence)
        decoded_sentence = self.final_layer_decoder(decoded_mid)
        
        print(decoded_sentence.shape)

In [ ]:
input_size = vocab_size # Size of embedding
hidden_size = 256 # Can set this to be a custom value
num_layers = 1 # Can change this
bs = 128
# vocab_size = len(vocab)
dropout = 0.0
bidirectional = False
max_sentence_length = 28 # Max number of words in a sentence
embedding_dims = 100


a = PGen(embedding_dims, hidden_size, dropout,vocab_size,input_size,num_layers)
a.forward(X_train_final,Y_train_final)

In [ ]:
##### DONT RUN THIS RIGHT NOW #### 
##### THIS IS THE OLD VERSION OF OUR PREPROCESSING STEPS ######
##### CHECK OUT THE CODE AT THE BOTTOM FIRST ######



with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_raw_train.json') as f:
    training_examples = json.load(f)
    
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_raw_val.json') as f:
    val_examples = json.load(f)

with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_data_prepro.json') as f:
    vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created   Vocab_inv = {'the':1, 'sauce':2, .....}

X_train_prelim = [training_examples[i]['question1'] for i in range(len(training_examples))] # Training examples before preprocessing
Y_train_prelim = [training_examples[i]['question'] for i in range(len(training_examples))]

X_val_prelim = [val_examples[i]['question1'] for i in range(len(val_examples))]
Y_val_prelim = [val_examples[i]['question'] for i in range(len(val_examples))]



def load_glove_embeddings(filename="glove.6B.100d.txt"):
    try:
        with open("/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/embeddings", "rb") as f:
            embeddings = pickle.load(f)
        return embeddings
    except:
        lines = open(filename).readlines()
        print(len(lines))
        embeddings = {}
        a = 0
        for line in lines:
            a +=1
            print(a)
            word = line.split()[0]
            embedding = list(map(float, line.split()[1:]))
            if word in vocab.values():
                embeddings[int(vocab_inv[word])] = embedding # This is creating a dictionary with indexes corresponding to the position of the particular word in the vocabulary
                # embeddings = {1:embedding of 'the', 2: embedding of 'sauce'....}
        return embeddings

def tokenize_embed(X,Y): # Doing this for training and validation_datasets
    X_train = []
    Y_train = []
    for i in range(len(X)): # This is basically tokenising each sentence, and converting it into a list of word embeddings. X corresponds to a training example and Y corresponds to its paraphrase
        print(i)
        sentence_x = X[i]
        sentence_y = Y[i]
        tokensx = word_tokenize(sentence_x.lower())
        tokensy = word_tokenize(sentence_y.lower())
        vectorx = []
        vectory = []
        for word in tokensx:
            try:
                vectorx.append(int(vocab_inv[word]))
            except:
                vectorx.append(258) # If the word is not in the embeddings given, then add a ',' instead. (Need to discuss this)
        for word in tokensy:
            try:
                vectory.append(int(vocab_inv[word]))
            except:
                vectory.append(258)
        X_train.append(vectorx)
        Y_train.append(vectory)
    return X_train,Y_train

def clip_pad_input_sequences(X,Y,min_sentence_length,max_sentence_length): # Doing this for training and validation datasets
    X_training_clipped = []
    Y_training_clipped = []
    for i in range(len(X)):
        print(i,X[i])
        if len(X[i]) >= min_sentence_length and len(X[i]) <= max_sentence_length and len(Y[i]) >= min_sentence_length and len(Y[i]) <= max_sentence_length:
            X_training_clipped.append(torch.Tensor(X[i]))
            Y_training_clipped.append(torch.Tensor(Y[i]))
    padded_X_training_clipped = pad_sequence(X_training_clipped,padding_value = vocab_size)
    padded_Y_training_clipped = pad_sequence(X_training_clipped,padding_value = vocab_size)
    return padded_X_training_clipped,padded_Y_training_clipped

def find_closest_embedding(output,vocab,embedded_words): # Need this because we probably need to find the closest embedding to a given embedding and thereby find the word
    return

min_sentence_length = 7
max_sentence_length = 16
vocab_size = len(vocab)
embedded_words = load_glove_embeddings()

X_train,Y_train = clip_pad_input_sequences(*tokenize_embed(X_train_prelim,Y_train_prelim,),min_sentence_length,max_sentence_length)
X_valid,Y_valid = clip_pad_input_sequences(*tokenize_embed(X_val_prelim,Y_val_prelim,),min_sentence_length,max_sentence_length)

In [ ]:
###### THIS IS THE PREPROCESSING THEY DID IN THE PAPER ####
##### RUN THIS IF U WANT AFTER CHANGING THE DIRECTORIES ######


def process_data(data, data_len):
    N = data.size()[0]
    new_data = torch.zeros(N, data.size()[1] + 2, dtype=torch.long) + PAD_token
    for i in range(N):
        new_data[i, 1:data_len[i]+1] = data[i, :data_len[i]]
        new_data[i, 0] = SOS_token
        new_data[i, data_len[i]+1] = EOS_token
        data_len[i] += 2
    return new_data, data_len

import h5py
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_data_prepro.json') as input_file:
    data_dict = json.load(input_file)

ix_to_word = {}

for k in data_dict['ix_to_word']:
    ix_to_word[int(k)] = data_dict['ix_to_word'][k]

UNK_token = 0

if 0 not in ix_to_word:
    ix_to_word[0] = '<UNK>'

else :
    raise Exception

EOS_token = len(ix_to_word)
ix_to_word[EOS_token] = '<EOS>'
PAD_token = len(ix_to_word)
ix_to_word[PAD_token] = '<PAD>'
SOS_token = len(ix_to_word)
ix_to_word[SOS_token] = '<SOS>'
vocab_size = len(ix_to_word)
print('DataLoader loading h5 question file:', '/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/quora_data_prepro.h5')
qa_data = h5py.File('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/quora_data_prepro.h5', 'r')

ques_id_train = torch.from_numpy(qa_data['ques_cap_id_train'][...].astype(int))

ques_train, ques_len_train = process_data(torch.from_numpy(qa_data['ques_train'][...].astype(int)), torch.from_numpy(qa_data['ques_length_train'][...].astype(int)))

label_train, label_len_train = process_data(torch.from_numpy(qa_data['ques1_train'][...].astype(int)), torch.from_numpy(qa_data['ques1_length_train'][...].astype(int)))

train_id = 0
seq_length = ques_train.size()[1]

print('Training dataset length : ', ques_train.size()[0])


ques_test, ques_len_test = process_data(torch.from_numpy(qa_data['ques_test'][...].astype(int)), torch.from_numpy(qa_data['ques_length_test'][...].astype(int)))

label_test, label_len_test = process_data(torch.from_numpy(qa_data['ques1_test'][...].astype(int)), torch.from_numpy(qa_data['ques1_length_test'][...].astype(int)))

ques_id_test = torch.from_numpy(qa_data['ques_cap_id_test'][...].astype(int))

test_id = 0

print('Test dataset length : ', ques_test.size()[0])
qa_data.close()

ques = torch.cat([ques_train, ques_test])
len = torch.cat([ques_len_train, ques_len_test])
label = torch.cat([label_train, label_test])
label_len = torch.cat([label_len_train, label_len_test])
id = torch.cat([ques_id_train, ques_id_test])





In [5]:
#### CAN PICKLE.DUMP THIS SHIZZZ #####
X_train,Y_train = ques_train,label_train
a = (X_train,Y_train)
X_valid,Y_valid = ques_test,label_test
b = (X_valid,Y_valid)
import pickle
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Training_Examples','wb') as f:
    pickle.dump(a,f)
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Validation_Examples','wb') as f:
    pickle.dump(b,f)
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Vocab_Extra','wb') as f:
    pickle.dump(ix_to_word,f)


vocab_size = len(vocab)  
print(X_train[0])
def one_hot(t, c):
    return torch.zeros(*t.size(), c, device=t.device).scatter_(-1, t.unsqueeze(-1), 1)
a = X_train[0]
b = one_hot(a, vocab_size)


tensor([27698, 21519, 16077,  7703,  8223, 23290, 25099, 22596, 17182, 27696,
        27697, 27697, 27697, 27697, 27697, 27697, 27697, 27697, 27697, 27697,
        27697, 27697, 27697, 27697, 27697, 27697, 27697, 27697])


In [6]:
##### RUN THIS TO SEE THE OUTPUT ####
##### THEY HAVE ADDED EXTRA TOKENS ####

import pickle
import torch
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Training_Examples','rb') as f:
    train = pickle.load(f)
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Validation_Examples','rb') as f:
    val = pickle.load(f)
with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Vocab_Extra','rb') as f:
    vocab = pickle.load(f)
X_train,Y_train = train[0],train[1]
print(len(X_train),X_train[0].shape[0],len(vocab))
X_train_final = torch.zeros(len(X_train),X_train[0].shape[0],len(vocab)) 
Y_train_final = torch.zeros(len(Y_train),Y_train[0].shape[0],len(vocab))
for i in range(128):
    print(i)
    a = one_hot(X_train[i], vocab_size) 
    print(a.shape)
    X_train_final[i,:,:] = one_hot(X_train[i], len(vocab))
    Y_train_final[i,:,:] = one_hot(Y_train[i], len(vocab))
# for i in range
vocab_size = len(vocab)

100000 28 27699
0
torch.Size([28, 27699])
1
torch.Size([28, 27699])
2
torch.Size([28, 27699])
3
torch.Size([28, 27699])
4
torch.Size([28, 27699])
5
torch.Size([28, 27699])
6
torch.Size([28, 27699])
7
torch.Size([28, 27699])
8
torch.Size([28, 27699])
9
torch.Size([28, 27699])
10
torch.Size([28, 27699])
11
torch.Size([28, 27699])
12
torch.Size([28, 27699])
13
torch.Size([28, 27699])
14
torch.Size([28, 27699])
15
torch.Size([28, 27699])
16
torch.Size([28, 27699])
17
torch.Size([28, 27699])
18
torch.Size([28, 27699])
19
torch.Size([28, 27699])
20
torch.Size([28, 27699])
21
torch.Size([28, 27699])
22
torch.Size([28, 27699])
23
torch.Size([28, 27699])
24
torch.Size([28, 27699])
25
torch.Size([28, 27699])
26
torch.Size([28, 27699])
27
torch.Size([28, 27699])
28
torch.Size([28, 27699])
29
torch.Size([28, 27699])
30
torch.Size([28, 27699])
31
torch.Size([28, 27699])
32
torch.Size([28, 27699])
33
torch.Size([28, 27699])
34
torch.Size([28, 27699])
35
torch.Size([28, 27699])
36
torch.Size([28, 276